In [1]:
import sys
sys.path.append('../rllab-curriculum-master/')
import curriculum.envs.arm3d.arm3d_key_env as key_env
import os.path as osp
import pickle
import numpy as np
import time
from curriculum.state.utils import StateCollection
from curriculum.envs.start_env import generate_starts
from rllab.envs.normalized_env import normalize
from curriculum.envs.goal_start_env import GoalStartExplorationEnv
from curriculum.envs.base import FixedStateGenerator
from rllab.misc.instrument import VariantGenerator
import random 

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/home/rabea/anaconda3/envs/rllab_goal_rl/lib/python3.5/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


In [14]:
vg = VariantGenerator()

vg.add('start_size', [7])
vg.add('start_goal', lambda start_size: [(1.55, 0.4, -3.75, -1.15, 1.81, -2.09, 0.05)] if start_size == 7 else
           [(1.55, 0.4, -3.75, -1.15, 1.81, -2.09, 0.05, 0, 0, 0, 0, 0, 0, 0)])
vg.add('goal_size', [9])
vg.add('ultimate_goal', [(0.0, 0.3, -0.7, 0.0, 0.3, -0.4, -0.15, 0.3, -0.55)])
vg.add('terminal_eps', [0.03])
vg.add('goal_size', [9])
vg.add('distance_metric', ['L2'])
vg.add('extend_dist_rew', [False])
vg.add('inner_weight', [0])
vg.add('goal_weight', lambda inner_weight: [1000] if inner_weight > 0 else [1])

print(vg.variants())

[{'distance_metric': 'L2', 'goal_size': 9, 'start_size': 7, 'goal_weight': 1, 'ultimate_goal': (0.0, 0.3, -0.7, 0.0, 0.3, -0.4, -0.15, 0.3, -0.55), '_hidden_keys': [], 'start_goal': (1.55, 0.4, -3.75, -1.15, 1.81, -2.09, 0.05), 'extend_dist_rew': False, 'inner_weight': 0, 'terminal_eps': 0.03}]


In [15]:
env = key_env.Arm3dKeyEnv(ctrl_cost_coeff=0)

start_state = lambda start_size: [(1.55, 0.4, -3.75, -1.15, 1.81, -2.09, 0.05)] if start_size == 7 else [(1.55, 0.4, -3.75, -1.15, 1.81, -2.09, 0.05, 0, 0, 0, 0, 0, 0, 0)]
    
fixed_start_generator = FixedStateGenerator(state=vg.variants()[0]['start_goal'])
fixed_goal_generator = FixedStateGenerator(state=vg.variants()[0]['ultimate_goal'])

inner_env = normalize(env)
start_goal_env = GoalStartExplorationEnv(
        env=inner_env,
        start_generator=fixed_start_generator,
        obs2start_transform=lambda x: x[:vg.variants()[0]['start_size']],
        goal_generator=fixed_goal_generator,
        obs2goal_transform=lambda x: x[-1 * vg.variants()[0]['goal_size']:],  # the goal are the last 9 coords
        terminal_eps=vg.variants()[0]['terminal_eps'],
        distance_metric=vg.variants()[0]['distance_metric'],
        extend_dist_rew=vg.variants()[0]['extend_dist_rew'],
        inner_weight=vg.variants()[0]['inner_weight'],
        goal_weight=vg.variants()[0]['goal_weight'],
        terminate_env=True,
    )
# start_goal_env.reset()

In [16]:
print('Current obs env:', env.get_current_obs())
print()
print('Current obs innver_env:', inner_env.get_current_obs())
print()
print('Current obs start_goal_env:', start_goal_env.get_current_obs())

Current obs env: [ 0.1         0.1        -1.54       -1.7         1.54       -0.2         0.
  0.          0.          0.          0.          0.          0.          0.
  0.35808759  0.40021986 -0.2249006   0.39271381  0.33405525  0.06565623
  0.34470621  0.50946443 -0.04355407]

Current obs innver_env: [ 0.1         0.1        -1.54       -1.7         1.54       -0.2         0.
  0.          0.          0.          0.          0.          0.          0.
  0.35808759  0.40021986 -0.2249006   0.39271381  0.33405525  0.06565623
  0.34470621  0.50946443 -0.04355407]

Current obs start_goal_env: [ 0.1         0.1        -1.54       -1.7         1.54       -0.2         0.
  0.          0.          0.          0.          0.          0.          0.
  0.35808759  0.40021986 -0.2249006   0.39271381  0.33405525  0.06565623
  0.34470621  0.50946443 -0.04355407  0.          0.3        -0.7         0.
  0.3        -0.4        -0.15        0.3        -0.55      ]


In [17]:
for i in range(5):
    print(env.step(action=np.array([0, .0, 0, 0.0, 0.0, 0.0, 1.0])))
    print(start_goal_env.start_observation)
    

Step(observation=array([  9.99946865e-02,   9.99798703e-02,  -1.54000041e+00,
        -1.69981234e+00,   1.53940944e+00,  -2.00008109e-01,
         1.71526437e-03,  -5.31352194e-04,  -2.01297416e-03,
        -4.05176616e-05,   1.87657461e-02,  -5.90556392e-02,
        -8.10874322e-04,   1.71526437e-01,   3.57898066e-01,
         4.00173028e-01,  -2.24882595e-01,   3.92838767e-01,
         3.34081640e-01,   6.56532480e-02,   3.44683689e-01,
         5.09456491e-01,  -4.35472561e-02]), reward=-12.925766328327672, done=False, info={})
[ 0.09999469  0.09997987 -1.54000041 -1.69981234  1.53940944 -0.20000811
  0.00171526]
Step(observation=array([  9.99838365e-02,   9.99418393e-02,  -1.53999960e+00,
        -1.69944313e+00,   1.53824680e+00,  -2.00022649e-01,
         5.10890714e-03,  -1.08499979e-03,  -3.80309639e-03,
         8.09737420e-05,   3.69211831e-02,  -1.16264835e-01,
        -1.45398740e-03,   3.39364277e-01,   3.57520804e-01,
         4.00079557e-01,  -2.24846010e-01,   3.930858

In [100]:
env.reset()

array([ 0.1       ,  0.1       , -1.54      , -1.7       ,  1.54      ,
       -0.2       ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.35808759,
        0.40021986, -0.2249006 ,  0.39271381,  0.33405525,  0.06565623,
        0.34470621,  0.50946443, -0.04355407])

In [120]:
with env.set_kill_outside(radius=0.5):
    print(env.step(action=np.random.uniform(env.action_space.bounds)))


Step(observation=array([  1.01547417e-01,   1.03023803e-01,  -1.51973088e+00,
        -1.69469985e+00,   1.51925382e+00,  -1.37585055e-01,
         3.66337834e-02,   2.89791974e-04,   1.18359424e-02,
         5.25020848e-02,  -1.77874524e-02,  -2.39996263e-01,
         2.00614523e-01,   1.20744383e-01,   3.57549539e-01,
         3.91683973e-01,  -2.29184592e-01,   3.94197313e-01,
         3.38675732e-01,   6.38121090e-02,   3.44722133e-01,
         5.08833711e-01,  -5.28003624e-02]), reward=-12.767421784493783, done=False, info={})


In [12]:
np.random.uniform(*env.action_space.bounds)

array([ 0.09064453,  0.55134477,  0.44003048, -0.48037999, -0.2101182 ,
       -0.33983504,  0.51311667])

In [7]:
load_dir = '../rllab-curriculum-master/data_upload/state_collections/'
old_all_feasible_starts = pickle.load(open(osp.join(load_dir, 'all_feasible_states.pkl'), 'rb'))

In [8]:
# print(env.kill_outside)
# print(env.kill_radius)

env.reset()
obs = env.get_current_obs()
print(obs)
print((obs == 0).sum())
print(obs[16])
print('Initialized with start state:')

init = old_all_feasible_starts.state_list[1]
# init.append(1)
print(np.array(init))
print()
print(env.reset(init_state=old_all_feasible_starts.state_list[0]))


[ 0.1         0.1        -1.54       -1.7         1.54       -0.2         0.
  0.          0.          0.          0.          0.          0.          0.
  0.35808759  0.40021986 -0.2249006   0.39271381  0.33405525  0.06565623
  0.34470621  0.50946443 -0.04355407]
8
-0.224900595763
Initialized with start state:
[ 1.54941984  0.40005018 -3.72492284 -1.16877705  1.80573215 -2.06306151
 -0.05838931]

[ 1.54952541  0.39858111 -3.74627502 -1.15193943  1.80397772 -2.08335436
  0.04660454  0.          0.          0.          0.          0.          0.
  0.          0.00637064  0.30922335 -0.68454249 -0.00446433  0.30021146
 -0.38487369 -0.14893819  0.30308163 -0.54017682]


In [15]:
env.reset()

array([ 0.1       ,  0.1       , -1.54      , -1.7       ,  1.54      ,
       -0.2       ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.35808759,
        0.40021986, -0.2249006 ,  0.39271381,  0.33405525,  0.06565623,
        0.34470621,  0.50946443, -0.04355407])

In [9]:
# in rllab/envs/mujoco/mujoco.py _get_full_obs() get_current_obs is defined
print('qpos:', env.model.data.qpos)
print()
print('qvel:', env.model.data.qvel)
print()
print('qacc:', env.model.data.qacc)
# # print('cinert shape:', env.model.data.cinert.shape)
# # print()
# # print('cvel:', env.model.data.cvel)
# # print()
# print('qfrc_actuator:', env.model.data.qfrc_actuator)
# print()
# # print('cfrc_ext:', env.model.data.cfrc_ext)
# # print()
# # print('qfrc_constraint:', env.model.data.qfrc_constraint)
# # print()
# # print('cdists:', env.model.geom_margin)
# # print()
# print('dcom:', env.dcom)

qpos: [[ 1.54952541]
 [ 0.39858111]
 [-3.74627502]
 [-1.15193943]
 [ 1.80397772]
 [-2.08335436]
 [ 0.04660454]]

qvel: [[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]

qacc: [[  8.90229372e-13]
 [ -3.00340019e-12]
 [ -1.50912239e-12]
 [ -6.38735709e-12]
 [  2.33598436e-12]
 [  1.23119397e-12]
 [  1.02165410e-13]]


In [13]:
def find_out_feasible_states_test(env, log_dir=None, distance_threshold=0.1, brownian_variance=1, animate=False):
    start_time = time.time()
    no_new_states = 0
    with env.set_kill_outside():
        load_dir = '../rllab-curriculum-master/data_upload/state_collections/'
        old_all_feasible_starts = pickle.load(open(osp.join(load_dir, 'all_feasible_states.pkl'), 'rb'))
        out_feasible_starts = StateCollection(distance_threshold=distance_threshold)
        print('number of feasible starts: ', old_all_feasible_starts.size)
        cnt = 0
        for start in old_all_feasible_starts.state_list:
            obs = env.reset(init_state=start)
#             print('Now start %i' %cnt)
            cnt += 1
            if cnt > 300:
                break
            if obs[16] > -0.5:
                # print("got one more up to ", out_feasible_starts.size)
                out_feasible_starts.append([start])
        print("number of out feasible starts:", out_feasible_starts.size)
#         while no_new_states < 5:
        total_num_starts = out_feasible_starts.size
        starts = out_feasible_starts.sample(100)
        new_starts = generate_starts(env, starts=vg.variants()[0]['start_goal'], horizon=10, variance=brownian_variance)# animated=animate, speedup=10, size=10)
        out_feasible_starts.append(new_starts)
        num_new_starts = out_feasible_starts.size - total_num_starts
        logger.log("number of new states: " + str(num_new_starts))
        if num_new_starts < 10:
            no_new_states += 1
#             with open(osp.join(log_dir, 'all_out_feasible_states.pkl'), 'wb') as f:
#                 cloudpickle.dump(out_feasible_starts, f, protocol=3)
    end_time = time.time()
    print('Time needed for start state generation: %.2fmin' %((end_time - start_time)/60))
    return out_feasible_starts

In [27]:
def find_out_feasible_states(env, log_dir=None, distance_threshold=0.1, brownian_variance=1, animate=False):
    start_time = time.time()
    no_new_states = 0
    with env.set_kill_outside():
        load_dir = '../rllab-curriculum-master/data_upload/state_collections/'
        old_all_feasible_starts = pickle.load(open(osp.join(load_dir, 'all_feasible_states.pkl'), 'rb'))
        out_feasible_starts = StateCollection(distance_threshold=distance_threshold)
        print('number of feasible starts: ', old_all_feasible_starts.size)
        cnt = 0
        for start in old_all_feasible_starts.state_list:
            obs = env.reset(init_state=start)
#             print('Now start %i' %cnt)
            cnt += 1
            if cnt > 300:
                break
            if obs[16] > -0.5:
                # print("got one more up to ", out_feasible_starts.size)
                out_feasible_starts.append([start])
        print("number of out feasible starts:", out_feasible_starts.size)
        while no_new_states < 5:
            total_num_starts = out_feasible_starts.size
            starts = out_feasible_starts.sample(100)
            new_starts = generate_starts(env, starts=starts, horizon=10, variance=brownian_variance)# animated=animate, speedup=10, size=10)
            out_feasible_starts.append(new_starts)
            num_new_starts = out_feasible_starts.size - total_num_starts
            logger.log("number of new states: " + str(num_new_starts))
            if num_new_starts < 10:
                no_new_states += 1
#             with open(osp.join(log_dir, 'all_out_feasible_states.pkl'), 'wb') as f:
#                 cloudpickle.dump(out_feasible_starts, f, protocol=3)
    end_time = time.time()
    print('Time needed for start state generation: %.2fmin' %((end_time - start_time)/60))
    return out_feasible_starts

In [14]:
new_starts = find_out_feasible_states_test(start_goal_env)

number of feasible starts:  136039
2019-01-21 20:00:21.746860 CET | we are trying to append states: (1, 7)
2019-01-21 20:00:21.747751 CET | after processing, we are left with : (1, 7)
2019-01-21 20:00:21.750673 CET | we are trying to append states: (1, 7)
2019-01-21 20:00:21.751836 CET | after processing, we are left with : (1, 7)
2019-01-21 20:00:21.756353 CET | we are trying to append states: (1, 7)
2019-01-21 20:00:21.758002 CET | after processing, we are left with : (1, 7)
2019-01-21 20:00:21.762102 CET | we are trying to append states: (1, 7)
2019-01-21 20:00:21.763713 CET | after processing, we are left with : (1, 7)
2019-01-21 20:00:21.768525 CET | we are trying to append states: (1, 7)
2019-01-21 20:00:21.771481 CET | after processing, we are left with : (1, 7)
2019-01-21 20:00:21.775969 CET | we are trying to append states: (1, 7)
2019-01-21 20:00:21.779127 CET | after processing, we are left with : (1, 7)
2019-01-21 20:00:21.783732 CET | we are trying to append states: (1, 7)

TypeError: 'float' object is not subscriptable

In [25]:
b = np.array([[[1,2],[3,4],[5,6]],[[7,8],[9,10],[11,12]]])
b.shape

(2, 3, 2)

In [59]:
def sample_n(array, n):
    ''' sample n elements from the array '''
    num_elements = array.shape[0]
    inds = np.random.choice(num_elements, size=n, replace=False)
    return array[inds,...]

sample_n(np.array([[2, 3], [1, 5],[1, 6],[1, 4],[1, 7],[1, 8],[1, 9]]), n=5)

array([[1, 8],
       [1, 9],
       [1, 7],
       [1, 4],
       [2, 3]])

In [63]:
rand_a = np.random.normal(size=start_goal_env.action_dim)
state, reward, done, obs_info = start_goal_env.step(rand_a)
print('State:', state)
print('reward:', reward)
print('done:', done)
print('obs_info:', obs_info)
start_goal_env.start_observation

State: [ -1.87090204e-04   1.78104648e-03   5.63863710e-03  -3.25645500e-03
  -2.06233387e-02  -5.03972428e-03   1.76960641e-02  -9.13712273e-03
   2.58631768e-02   1.17504747e-01  -7.32714031e-02  -5.50233942e-01
  -2.04712110e-02   5.42521905e-01   1.00262388e+00  -1.87505331e-01
  -2.49060001e-01   1.00066944e+00  -1.88318393e-01   5.09325309e-02
   1.15164348e+00  -1.87931352e-01  -9.80865646e-02   0.00000000e+00
   3.00000000e-01  -7.00000000e-01   0.00000000e+00   3.00000000e-01
  -4.00000000e-01  -1.50000000e-01   3.00000000e-01  -5.50000000e-01]
reward: 0.0
done: False
obs_info: {'distance': 2.2418688175841259, 'reward_dist': 0.0, 'goal': (0.0, 0.3, -0.7, 0.0, 0.3, -0.4, -0.15, 0.3, -0.55), 'reward_inner': -0.0, 'goal_reached': 0.0}


array([-0.00018709,  0.00178105,  0.00563864, -0.00325645, -0.02062334,
       -0.00503972,  0.01769606])

In [76]:
def sample_n(array, n):
    ''' sample n elements from the array '''
    num_elements = array.shape[0]
    inds = np.random.choice(num_elements, size=n, replace=False)
    return array[inds, ...]


def sample_nearby(env, states, n_new=200, variance=0.9, t_b=50, M=1000):
    '''
        n_new: number of sampled starts in the ned
        t_b:   horizon of one trajectory
        M:     number of state list after adding starts

    '''
    # possibly add: radius which tests if sampled states are in a given
    # radius around the sampled start state or around the goal?

    starts = np.array(states)
    while(len(starts) < M):
        # number of starts given by first dimension of starts
        # num_starts = starts.shape[0]
        # s_0 = starts[np.random.choice(num_starts),...]
        s_0 = random.choice(starts)
        env.reset(init=s_0)
        for i in range(t_b):
            a = np.random.normal(scale=variance, size=env.action_dim)
            # only want the current observation
            _obs, _rew, _done, _env_info = env.step(a)
            # only want the start state part of the current state
            s_1 = env.start_observation.reshape(1,-1)
            starts = np.append(starts, s_1, axis=0)
            # rllab also tested if the exploration led through the goal state
            # do not see why this is necessary atm

    # now sample from the M start states
    new_starts = sample_n(starts, n_new)
    return new_starts


In [78]:
new_starts = sample_nearby(start_goal_env, states=[start_goal_env.start_observation])
new_starts.shape

(200, 7)